In [1]:
import pandas as pd

In [2]:
file_path = "/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/raw/balancing_market/fcr/RESULT_OVERVIEW_CAPACITY_MARKET_FCR_2024-01-01_2024-12-31.xlsx"

In [12]:
df = pd.read_excel(file_path)

/home/kehan-vm/miniconda3/envs/env_power_market_analysis/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [13]:
df.head()

,DATE_FROM,DATE_TO,PRODUCT_TYPE,TENDER_NUMBER,PRODUCTNAME,CROSSBORDER_SETTLEMENTCAPACITY_PRICE_[EUR/MW],AUSTRIA_DEMAND_[MW],AUSTRIA_SETTLEMENTCAPACITY_PRICE_[EUR/MW],AUSTRIA_DEFICIT(-)_SURPLUS(+)_[MW],BELGIUM_DEMAND_[MW],...,NETHERLANDS_DEFICIT(-)_SURPLUS(+)_[MW],SLOVENIA_DEMAND_[MW],SLOVENIA_SETTLEMENTCAPACITY_PRICE_[EUR/MW],SLOVENIA_DEFICIT(-)_SURPLUS(+)_[MW],SWITZERLAND_DEMAND_[MW],SWITZERLAND_SETTLEMENTCAPACITY_PRICE_[EUR/MW],SWITZERLAND_DEFICIT(-)_SURPLUS(+)_[MW],CZECH_REPUBLIC_DEMAND_[MW],CZECH_REPUBLIC_SETTLEMENTCAPACITY_PRICE_[EUR/MW],CZECH_REPUBLIC_DEFICIT(-)_SURPLUS(+)_[MW]
0,2024-01-01,2024-01-01,FCR,1,NEGPOS_00_04,114.8,72,114.8,11,93,...,-31,14,114.8,-7,62,114.8,-20,77,416,-51
1,2024-01-01,2024-01-01,FCR,1,NEGPOS_04_08,104.4,72,104.4,11,93,...,-28,14,104.4,-7,62,104.4,-22,77,416,-51
2,2024-01-01,2024-01-01,FCR,1,NEGPOS_08_12,68.8,72,68.8,22,93,...,-42,14,68.8,-13,62,68.8,-21,77,416,-51
3,2024-01-01,2024-01-01,FCR,1,NEGPOS_12_16,77.6,72,77.6,11,93,...,-28,14,77.6,-13,62,77.6,-20,77,390.4,-51
4,2024-01-01,2024-01-01,FCR,1,NEGPOS_16_20,72.4,72,72.4,20,93,...,-42,14,72.4,-14,62,72.4,-21,77,390.4,-51


In [14]:
# Extract hour from PRODUCTNAME using string slicing
df['hour'] = df['PRODUCTNAME'].str[7:9].astype(int)

# keep only fisrt tender (tender_number = 1)
df = df[df['TENDER_NUMBER'] == 1]

# Combine DATE_FROM with hour to create datetime
df['datetime'] = pd.to_datetime(df['DATE_FROM']) + pd.to_timedelta(df['hour'], unit='h')

# check if there are any duplicates
if df['datetime'].duplicated().any():
    print('There are duplicates in the datetime column')
    # Show all rows that have duplicate datetime values
    duplicate_rows = df[df[['datetime', 'PRODUCTNAME']].duplicated(keep=False)].sort_values('datetime')
    print("\nRows with duplicate datetimes:")
    print(duplicate_rows)
    # Remove duplicates while preserving the first occurrence
    df = df.drop_duplicates(subset='datetime', keep='first')
else:
    print('There are no duplicates in the datetime column')
# check missing timestamps
complete_timestamps = pd.date_range(start=df['datetime'].min(), end=df['datetime'].max(), freq='4H')
missing_timestamps = complete_timestamps[~complete_timestamps.isin(df['datetime'])]
if len(missing_timestamps) > 0:
    print(f"Missing timestamps: {missing_timestamps}")
else:
    print('No missing timestamps')

There are no duplicates in the datetime column
No missing timestamps


/tmp/ipykernel_119082/1066069491.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_timestamps = pd.date_range(start=df['datetime'].min(), end=df['datetime'].max(), freq='4H')


In [15]:
df.to_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/processed/balancing_market/fcr/fcr_price_2024.csv', index=False)